Import the repo

In [ ]:
!git clone https://github.com/username/yolov1-from-scratch.git

%cd yolov1-from-scratch

from src.yolo_model import YOLOv1
from src.loss import yolo_loss
from src.dataloader import YoloDataset

Switch to GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('device:', device)

Model Config
C = number of cards = 52 

In [ ]:
config = {
    'S': 7,
    'B': 2,
    'C': 52
    'batch_size': 32,
    'lambda_coord': 5,
    'lambda_noobj': 0.5
}

Create the model

In [ ]:
model = YOLOv1(config)
model = model.to(device)

Import the data using a dataloader

In [ ]:
data_dir = r'/kaggle/input/playing-cards-yolov1/train'

dataset = YoloDataset(data_dir, config)

example_img, example_label = dataset.__getitem__(0)
model.draw_output(example_img, example_label, t=0.0, transform=False)

Optimizer config

In [ ]:
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-3,
    weight_decay=5e-4
)

Train loop

In [ ]:
epochs = 100

dataloader = DataLoader(dataset, config['batch_size'], shuffle=True, num_workers=4, pin_memory=True)

model.train()

for epoch in range(epochs):
    for images_batch, labels_batch in dataloader:
        images_batch = images_batch.to(device)
        labels_batch = labels_batch.to(device)
        
        output_batch = model.forward(images_batch)
        loss = model.yolo_loss(output_batch, labels_batch, lambda_coord=5, lambda_noobj=0.5)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")